In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rc('font', size=15)

In [29]:
from jax import config
from jax import random
import jax.numpy as jnp
from jax.lax import fori_loop
from jax import jit

config.update('jax_enable_x64', True)

In [34]:
def one_loop(i, key):
    key, subkey = random.split(key, 2)
    lam = jnp.abs(random.normal(subkey, (1000, 1000)))
    n = random.poisson(subkey, lam, (1000, 1000))
    return key
    
key = random.PRNGKey(0)
%timeit fori_loop(0, 100, one_loop, key).block_until_ready()

4 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
def one_loop():
    lam = np.abs(np.random.normal(size=(1000,1000)))
    n = np.random.poisson(lam=lam, size=(1000, 1000))
    
%timeit [one_loop() for i in range(100)]

7.41 s ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [248]:
N = 10
n = jnp.zeros(N, dtype=jnp.int64)
n = n.at[0].set(10)
n = n.at[1].set(10)

obs_front = jnp.zeros(1, dtype=jnp.int64)
inn_front = jnp.ones(1, dtype=jnp.int64)
in_sub_pop = jnp.array([0,1])
sites = jnp.arange(N, dtype=jnp.int64)
Ady = jnp.eye(N, k=1, dtype=jnp.bool_)

inverse_sons = Ady.sum(1)
inverse_sons = inverse_sons.at[~inverse_sons].set(1)
inverse_sons = 1 / inverse_sons

r = 1
rd = .1
I = 1
G_in = 1
dt = .1
τo = 1

def move_innovation_ant(key, inn_front, in_sub_pop):
    key, subkey = random.split(key)
    
    old_inn_front = inn_front.copy()
    to_innov = random.uniform(subkey, (inn_front.size,)) < r*I*dt*n[inn_front]
    n_growing_sites = to_innov.sum()
    
    if n_growing_sites:
        new = jnp.zeros(n_growing_sites, dtype=jnp.int64)
        key, *subkeys = random.split(key, n_growing_sites+1)
        for i, ix in enumerate(inn_front[to_innov]):
            new = new.at[i].set(random.choice(subkeys[i], sites[Ady[ix]]))
        
        inn_front = jnp.concatenate((inn_front, new))
        inn_front = jnp.unique(inn_front)
        in_sub_pop = jnp.concatenate((in_sub_pop, new))
        in_sub_pop = jnp.unique(in_sub_pop)
    inn_front = jnp.setdiff1d(inn_front, old_inn_front[to_innov])  # is it unnecessary?
    
    return key, inn_front, in_sub_pop

"""
Update the innovation front
"""
key, inn_front, in_sub_pop = move_innovation_ant(key, inn_front, in_sub_pop)

"""
Replication 
"""
key, subkey = random.split(key)
to_replicate = random.poisson(subkey, (r * inverse_sons * n * dt) @ Ady)
n = n.at[in_sub_pop].set(n[in_sub_pop] + to_replicate[in_sub_pop])

"""
Death
"""
key, subkey = random.split(key)
to_died = random.poisson(subkey, rd * n * dt)
n = n - to_died                            
n = n.at[n<0].set(0)

"""
Growth
"""
key, subkey = random.split(subkey)
G_dt = random.poisson(subkey, G_in*dt/in_sub_pop.size, (in_sub_pop.size,))
n = n.at[in_sub_pop].set(n[in_sub_pop] + G_dt)

"""
Update obsolescence front 
"""
obs_not_inn = jnp.setdiff1d(obs_front, inn_front)

key, subkey = random.split(key)
to_obsolete = random.uniform(subkey, (obs_not_inn.size,)) < τo*dt
to_obsolete = obs_not_inn[to_obsolete]
obs_front = jnp.setdiff1d(obs_front, to_obsolete)
in_sub_pop = jnp.setdiff1d(in_sub_pop, to_obsolete)

to_add_obs = sites[Ady[to_obsolete].sum(0)>0]
to_add_obs = jnp.intersect1d(to_add_obs, in_sub_pop)
obs_front = jnp.concatenate((obs_front, to_add_obs))                            
obs_front = jnp.unique(obs_front)

In [249]:
inn_front, obs_front, n

(Array([2], dtype=int64),
 Array([0], dtype=int64),
 Array([10, 11,  1,  0,  0,  0,  0,  0,  0,  0], dtype=int64))